In [0]:
#Import librairies
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#for loading and visualizing audio files
import librosa
import librosa.display

#Import Drive
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Fonction "empreinte" et "lissage"

In [0]:
### Empreinte basé sur les maximums d'amplitude
# paramètre en entrée : signal sonore
def amplitude(i):
  I = librosa.stft(i) # Transormée de fourier (matrice : temps x fréquence)
  Idb = librosa.amplitude_to_db(abs(I)) # "Module" de la transformée 
  max_ampl = []
  for i in range(Idb.shape[1]):
    max_ampl.append(Idb[:,i].max())
  return(max_ampl)

### fonction de lissage (moyenne glissante)
def lissage(Ly,p):
  '''Fonction qui débruite une courbe par une moyenne glissante sur 2P+1 points'''
  Lyout=[]
  for index in range(p, len(Ly)-p):
    average = sum(Ly[index-p : index+p+1]) / (2*p + 1)
    Lyout.append(average)
          
  return Lyout

# Création de la base d'empreintes

In [11]:
### Création de la bibliothèque "amplitude"
meta = pd.read_csv ("/content/drive/My Drive/Projet final - Python/Livrables/metadata.csv", sep = ";", header=0)

dir_path = '/content/drive/My Drive/Projet final - Python/Livrables/Musicotheque/'
dico_empreinte = {}
for element in os.listdir(dir_path):
  file_path = dir_path + element
  x, sr = librosa.load(file_path)
  
  num = int(element[-7:-4])
  cle_df = meta.loc[meta['track_id']==num, ]
  cle = ""
  for i in range(0 , cle_df.shape[1]) :
    cle = cle + " - " + cle_df.columns[i] + " : " + str(cle_df.iloc[0,i])
  
  x_empreinte = amplitude(x)
  x_empreinte_lisse = lissage(lissage(x_empreinte, 2), 2)

  dico_empreinte[cle] = x_empreinte_lisse

print(dico_empreinte)

{' - track_id : 2 - artist_name : AWOL - album_title : AWOL - A Way Of Life - track_title : Food - track_date_created : 11/26/2008': [22.37137107849121, 23.67424362182617, 24.922151641845705, 26.26255844116211, 27.452218704223633, 28.43617973327637, 29.15864364624024, 29.81955993652344, 30.040287857055667, 29.94906013488769, 29.820559921264646, 29.727472763061524, 29.815857162475584, 30.770311813354493, 32.56321174621582, 34.82940956115723, 36.959387893676755, 38.25969306945801, 38.3202613067627, 37.293898315429686, 35.56088218688965, 33.66664932250977, 32.307708206176756, 31.66366241455078, 31.550747909545898, 31.734705505371096, 32.1291854095459, 32.39006996154785, 32.40927902221679, 32.40151496887207, 32.48029357910156, 32.60930091857911, 32.84708099365234, 33.14300956726074, 33.34893081665039, 33.41459678649902, 33.41637756347656, 33.33256378173828, 33.25789688110352, 33.20530838012695, 33.13570617675781, 32.985283355712895, 32.876336669921876, 32.83773696899415, 32.89066116333007,

# Reconnaissance musicale

### Comparaison de deux empreintes de longueur quelconque

In [0]:
# fonction qui compare deux empreintes
  def correl(x, y):
    if len(x) <= len(y) :
      extrait = x
      ref = y
    else :
      extrait = y
      ref = x

    borne_inf = 0
    max_coef = -1
    while borne_inf <= (len(ref) - len(extrait)):
      ref_court = ref[borne_inf : (borne_inf + len(extrait))]
      max_coef = max(max_coef, np.corrcoef(extrait, ref_court)[0,1])
      if max_coef == np.corrcoef(extrait, ref_court)[0,1]:
        instant = borne_inf
      borne_inf = borne_inf + 1

    return(max_coef, instant)

### Comparaison à la base de sons

In [0]:
# Entrée : path_sound = lien vers l'enregistrement
def recognize(path_sound):

  x, sr = librosa.load(path_sound)
  x = x[22050:] # on supprime la première seconde 
  x_empreinte = amplitude(x)
  x_empreinte_lisse = lissage(lissage(x_empreinte, 2), 2)

  # boucle de comparaison entre l'empreinte du signal d'entrée et la bibliothèque de sons
  max_biblio = -1
  for cle,valeur in dico_empreinte.items():
    cor, inst = correl(x_empreinte_lisse, valeur)
    max_biblio = max(max_biblio, cor)
    if max_biblio == cor :
        ref = cle
        sec = inst / 30
      
  print("Similitude : ", max_biblio)
  if max_biblio > 0.6 :
    print("L'extrait écouté correspond au titre : ")
    print(ref)
    print("À l'instant (s) : ", sec)
  elif max_biblio > 0.4 :
    print("Le morceau le plus proche :")
    print(ref)
  else :
    print("Aucune correspondance n'a été établie.")

# Test
### Avec un morceau original

In [15]:
path_test = '/content/drive/My Drive/Projet final - Python/Livrables/Musicotheque/000255.mp3'
recognize(path_test)

Similitude :  0.9999145833206274
L'extrait écouté correspond au titre : 
 - track_id : 255 - artist_name : Bird Names - album_title : On Opaque Things - track_title : We Want to be Old - track_date_created : 11/26/2008
À l'instant (s) :  1.4333333333333333


### Avec des sons enregistrés au micro

In [16]:
path_test = '/content/drive/My Drive/Projet final - Python/Livrables/Sons_micro/255.mp3'
recognize(path_test)

Similitude :  0.7400981384195056
L'extrait écouté correspond au titre : 
 - track_id : 255 - artist_name : Bird Names - album_title : On Opaque Things - track_title : We Want to be Old - track_date_created : 11/26/2008
À l'instant (s) :  2.1666666666666665


In [17]:
path_test = '/content/drive/My Drive/Projet final - Python/Livrables/Sons_micro/621.mp3'
recognize(path_test)

Similitude :  0.7315746090400717
L'extrait écouté correspond au titre : 
 - track_id : 621 - artist_name : Ed Askew - album_title : Rainy Day Song - track_title : Blue Eyed Baby - track_date_created : 11/26/2008
À l'instant (s) :  8.533333333333333
